by Edouard Ratiarson
March 21st 2024

# Music Generation with Chord Accompaniment

This code is an implementation of a music generation system with chord accompaniment.

This code demonstrates how to extend a recurrent neural network (RNN) and LSTM-based music generation model to include chord accompaniment that harmonizes with a given melody. We will start by training a model on a collection of piano MIDI files from the Maestro dataset. The model will learn to predict the next note in a sequence, forming a melody. Subsequently, we will algorithmically generate chords that harmonically support the melody, focusing on fitting within the key and enhancing the overall musical piece.

1. **Setup and Importing Libraries**: The code begins by installing the required Python libraries such as `pretty_midi`, `tensorflow`, `pandas`, `matplotlib`, and `seaborn`. These libraries are essential for processing MIDI files, building neural networks, data manipulation, and visualization.

2. **MIDI File Processing**: The code defines several functions to extract melody and chord information from MIDI files. The `midi_to_melody_and_chords` function processes a MIDI file and separates the melody notes from the chord notes based on their pitch and start times. It returns two pandas DataFrames containing the extracted melody and chord data.

3. **Data Preprocessing**: The code includes functions to preprocess the extracted melody and chord data. The `normalize_pitches` function normalizes the pitch values to a range between 0 and 1, which is a common practice in machine learning for music data. The `create_sequences` function generates sequences of chords with a fixed length, suitable for training a sequence prediction model.

4. **Dataset Handling**: The code includes a script to automatically download and extract the Maestro v2.0.0 dataset, a collection of piano MIDI files. This dataset is used for training the music generation model.

5. **Melody and Chord Extraction**: The code iterates through the Maestro dataset, extracting melody and chord sequences from each MIDI file. It also analyzes the extracted data, calculating the percentage of zero values (silent timesteps) in the chord sequences, which can be helpful for model tuning and training efficiency.

6. **One-hot Encoding**: The code defines a function called `to_one_hot` that converts sequences of chord indices into one-hot encoded vectors. This encoding is crucial for categorical classification tasks, allowing the model to treat each chord type as a distinct category.

7. **Neural Network Model**: The code sets up a neural network model in TensorFlow for predicting chords based on a given melody. It calculates the number of unique chords in the dataset and defines a sequential model with LSTM layers and a final dense layer with a softmax activation function for chord classification.

8. **Model Training**: The code prepares the targets (chords) for training and demonstrates how to train the neural network model using the preprocessed melody and chord sequences.

9. **Visualization**: The code includes a section for visualizing the training loss over epochs, which is a useful diagnostic tool for monitoring the model's learning progress.

10. **Chord Suggestion and Music Generation**: The code defines functions for suggesting chords for a given melody using the trained model. It also includes functions for generating music by predicting chords based on a seed sequence of melody notes.

Overall, this code implements a complete pipeline for music generation with chord accompaniment, including data preprocessing, model training, and inference for chord prediction and music generation.






## Setup

- Import necessary libraries and modules.
- Define any required constants or global variables for the project.


In [ ]:
# Install the required libraries
!pip install pretty_midi
!pip install tensorflow
!pip install pandas
!pip install matplotlib
!pip install seaborn

# After installation, you can import the libraries as before
import tensorflow as tf
import numpy as np
import pretty_midi
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

# Constants
SAMPLING_RATE = 16000  # Sampling rate for audio playback

print("Setup complete. Ready to process dataset and generate music.")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 1.5 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592289 sha256=3bf96646348b2668f248a4b757578f4251dd00816d3d8e1b2064f3509d7c03f4
  Stored in directory: /root/.cache/pip/wheels/cd/a5/30/7b8b7f58709f5150f67f98fde4b891ebf0be9ef07a8af49f25
Successfully built pretty_midi
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
Setup complete. Ready to process dataset and generate music.


This code block defines the `midi_to_melody_and_chords` function, which processes a MIDI file to separate the melody from the chords. It utilizes the `pretty_midi` library to read and analyze the MIDI file. The function iterates through all the notes in the file, sorting them by their start times to maintain temporal order.

For each note, the function checks whether it belongs to the current chord or starts a new one. If a note starts after the current chord's latest note, it concludes the chord, and the highest note of this chord is considered part of the melody, while the rest are considered as chords. This heuristic follows the common musical practice where the melody is often the highest note in the harmonic structure.

The extracted melody and chord notes are then converted into pandas DataFrames with columns 'Pitch', 'Start', and 'End', making it easier to manipulate and analyze the musical data further. The function returns these DataFrames, providing a structured representation of the melody and chords extracted from the MIDI file.


In [ ]:
import pretty_midi

def midi_to_melody_and_chords(midi_file: str):
    """Processes a MIDI file to extract melody and chords.

    Args:
        midi_file: Path to the MIDI file.

    Returns:
        A tuple of:
            * melody_data: A DataFrame with columns 'Pitch', 'Start', 'End'
            * chord_data: A DataFrame with columns 'Pitch', 'Start', 'End'
    """

    pm = pretty_midi.PrettyMIDI(midi_file)

    all_notes = [note for instrument in pm.instruments for note in instrument.notes]
    all_notes.sort(key=lambda note: note.start)

    melody_notes = []
    chord_notes = []

    current_time = 0
    current_chord = []

    for note in all_notes:
        if note.start > current_time:
            if current_chord:
                # Assume the highest note in the chord is the melody
                highest_note = max(current_chord, key=lambda n: n.pitch)
                melody_notes.append(highest_note)
                chord_notes.extend([n for n in current_chord if n != highest_note])

            current_chord = []
            current_time = note.end

        current_chord.append(note)

    # Process any remaining notes in the last chord
    if current_chord:
        highest_note = max(current_chord, key=lambda n: n.pitch)
        melody_notes.append(highest_note)
        chord_notes.extend([n for n in current_chord if n != highest_note])

    # Convert notes to DataFrames
    melody_data = pd.DataFrame([(n.pitch, n.start, n.end) for n in melody_notes], columns=['Pitch', 'Start', 'End'])
    chord_data = pd.DataFrame([(n.pitch, n.start, n.end) for n in chord_notes], columns=['Pitch', 'Start', 'End'])

    return melody_data, chord_data


This code block is designed to extract melody and chord information from a MIDI file. It uses the `pretty_midi` library to parse the MIDI file and extract note events. The `midi_to_melody_and_chords` function processes each note in the MIDI file to separate melody and chord notes based on their pitch and start times.

- The melody is determined by selecting the highest note at each point in time, based on the assumption that the melody note is typically the highest note in a chord or musical phrase.
- Chord notes are collected separately and are identified by their simultaneous occurrence and harmony with the melody.
- The extracted melody and chord notes are then converted into pandas DataFrames, `melody_df` and `chord_df`, for easier data manipulation and analysis. These DataFrames contain columns for pitch, start time, and end time of each note.

Additional functions like `get_melody` and `identify_chord` are defined to support the extraction process:
- `get_melody` selects the highest note (melody) from a set of chord notes.
- `identify_chord` attempts to name chords based on the intervals between the notes in a chord, specifically identifying major and minor triads.

This code is fundamental for analyzing the musical structure of the MIDI file and preparing the data for further musicological analysis or for training machine learning models to generate music based on the extracted patterns.


In [ ]:
import pretty_midi
import pandas as pd

import pretty_midi


max_duration = 5.0

def get_melody(chord_notes):
    """Selects the highest note as the melody."""
    if chord_notes:
        return max(chord_notes)  # Return the highest pitch
    else:
        return None  # Handle case when no notes are present


def midi_to_melody_and_chords(midi_file_path):
    """Extracts melody and chords from a MIDI file.

    Args:
        midi_file_path: The path to the MIDI file.

    Returns:
        A tuple: (melody_df, chord_data)
        * melody_df:  A Pandas DataFrame containing the extracted melody notes.
        * chord_data: A list of extracted chords.
    """

    pm = pretty_midi.PrettyMIDI(midi_file_path)

    all_melodies = []
    all_chords = []
    melody_notes = []
    chord_notes = []

    for instrument in pm.instruments:
        notes = instrument.notes
        notes.sort(key=lambda note: (note.start, -note.pitch))

        for note in notes:
            is_melody = note == notes[0]
            if is_melody:
                melody_notes.append((note.pitch, note.start, note.end))
            else:
                chord_notes.append(note.pitch)

    # Convert melody notes to a DataFrame
    melody_df = pd.DataFrame(melody_notes, columns=["Pitch", "Start", "End"])
    chord_df = pd.DataFrame(chord_notes, columns=["Pitch"])  # Or adjust the column name

    return melody_df, chord_df  # Return both as DataFrames




def process_midi(file_path):
    pm = pretty_midi.PrettyMIDI(file_path)

    melody_notes = []
    chord_notes = []

    for instrument in pm.instruments:
        notes = instrument.notes
        notes.sort(key=lambda note: (note.start, -note.pitch))  # Sort by start time, then highest pitch

        for note in notes:
            is_melody = note == notes[0]  # Check if it's the highest note at that given start time
            if is_melody:
                melody_notes.append((note.pitch, note.start, note.end))
            else:
                chord_notes.append(note.pitch)

    for instrument in pm.instruments:
        notes = instrument.notes
        notes.sort(key=lambda note: note.start)  # Sort by start time only

        time_step = 0.0
        while time_step < max_duration:  # Assuming you have a max_duration
            chord_notes = [note.pitch for note in notes if note.start <= time_step < note.end]
            chord = identify_chord(chord_notes)  # You'll need to implement this
            all_chords.append(chord)
            time_step += 0.25  # Or some other time increment

    new_melody_data = [(note, 0.0, 0.0) for note in melody_data]  # Placeholder for start and end

    print("Shape of melody_data:", np.shape(melody_data))
    print("First five elements of melody_data:", melody_data[:5])

    melody_df = pd.DataFrame(new_melody_data, columns=["Pitch", "Start", "End"])
    return melody_notes, chord_notes


def identify_chord(chord_notes):
    """Identifies basic major and minor triads based on the given notes."""

    if len(chord_notes) != 3:
        return "Unknown"  # Only handling triads for now

    # Calculate intervals between notes (modulo 12 for octaves)
    intervals = [(chord_notes[i] - chord_notes[0]) % 12 for i in range(1, 3)]

    # Check for major or minor triad patterns
    if intervals == [4, 7]:  # Major triad
        root_note = chord_notes[0] % 12  # Determine root note (modulo 12 for octave)
        return f"{note_names[root_note]} Major"
    elif intervals == [3, 7]:  # Minor triad
        root_note = chord_notes[0] % 12
        return f"{note_names[root_note]} Minor"
    else:
        return "Unknown"

# Helper: A list to map note numbers (0-11) to note names
note_names = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]


# Assuming you have your MIDI file path set correctly
basic_midi_file = '/content/basic_midi_file.midi'

# Load the MIDI file and extract melody and chords
melody_data, chord_data = midi_to_melody_and_chords(basic_midi_file)

# Assuming your melody_data is a list of notes represented as numbers or tuples
melody_df = pd.DataFrame(melody_data, columns=["Pitch", "Start", "End"])


# Print summaries of extracted melody and chord data
print("Melody Data:")
print(melody_data.head())  # Prints the first few rows of the melody DataFrame

print("\nChord Data:")
print(chord_data.head())  # Prints the first few rows of the chord DataFrame


FileNotFoundError: [Errno 2] No such file or directory: '/content/basic_midi_file.midi'

The function `create_sequences` takes two parameters: `chord_data`, which is a list of chords (each represented as a string), and `seq_length`, which specifies the length of each sequence to be created. The process involves several key steps:

- **Unique Chord Identification**: The function first identifies all unique chords in the dataset and sorts them. This step is crucial for creating a consistent mapping from chord strings to integers, which is needed for numerical processing in machine learning models.
- **Integer Encoding**: Each chord in the dataset is then mapped to an integer based on the sorted list of unique chords. This integer encoding converts the string representations into a format that can be efficiently processed by neural networks.
- **Sequence Creation**: The function iterates through the integer-encoded chord data, creating overlapping subsequences of the specified length (`seq_length`). Each subsequence serves as an input instance for the model, allowing it to learn patterns and dependencies in the chord progressions.

The output of the function is a NumPy array of these sequences, ready to be used in the training process of a sequence prediction model, such as an LSTM (Long Short-Term Memory) neural network. This preparation is essential for enabling the model to learn the structure and progression of chords in music.


In [ ]:
import numpy as np

def create_sequences(chord_data, seq_length):
    """Assumes 'chord_data' is a list of chords represented as strings"""

    unique_chords = sorted(list(set(chord_data)))
    chord_to_int = dict((c, i) for i, c in enumerate(unique_chords))

    # Integer Encoding
    int_encoded_data = [chord_to_int[chord] for chord in chord_data]

    # Create sequences
    sequences = []
    for i in range(len(int_encoded_data) - seq_length + 1):
        seq = int_encoded_data[i:i + seq_length]
        sequences.append(np.array(seq))

    return np.array(sequences)


The `normalize_pitches` function is designed to normalize MIDI pitch values to a range between 0 and 1, a common preprocessing step in machine learning for music data. This normalization helps in standardizing the input data range, making the training process more stable and efficient. The function can handle different data structures:

- **DataFrame Handling**: If the input `data` is a pandas DataFrame, the function assumes there is a 'Pitch' column and normalizes these values by dividing each by the maximum MIDI pitch value, 127. This operation is performed in-place, directly modifying the 'Pitch' column of the DataFrame.
- **List Handling**: If `data` is a list, the function checks if it contains integers (representing MIDI pitches) or tuples (representing notes with pitch, start, and end times). For a list of integers, each pitch is normalized individually. For a list of tuples, only the pitch element of each tuple is normalized, preserving the timing information.
- **Type Checking**: The function includes a type check to ensure that the input `data` is either a list or a DataFrame. If another data type is passed, a `TypeError` is raised to alert the user to the invalid input type.

This normalization process is crucial for maintaining consistency across various pieces of music and ensuring that the machine learning model treats all pitch values on the same scale.


In [ ]:
import pandas as pd

def normalize_pitches(data):
    """Normalizes MIDI pitches within a list or DataFrame

    Args:
        data:  A list of MIDI pitches (or tuples representing notes), or a DataFrame with a 'Pitch' column.

    Returns:
        The modified data (list or DataFrame) with normalized pitches.
    """
    if isinstance(data, pd.DataFrame):
        data['Pitch'] /= 127.0
    elif isinstance(data, list):
        if all(isinstance(item, int) for item in data):  # All elements are integers (pitches)
            data = [pitch / 127.0 for pitch in data]
        else:  # List of tuples
            data = [(pitch / 127.0, start, end) for (pitch, start, end) in data]
    else:
        raise TypeError("Data must be a list or a DataFrame")

    return data




This script is responsible for automatically downloading and extracting the Maestro v2.0.0 dataset, a widely used collection of piano MIDI files for music generation research. The steps executed by this script are:

1. **Define Dataset URL**: The URL for the Maestro dataset (`https://storage.googleapis.com/magentadata/datasets/maestro/v2.0.0/maestro-v2.0.0-midi.zip`) is stored in the `url` variable. This dataset is hosted by Google Cloud Storage and is publicly accessible.

2. **Download the Dataset**: Using the `requests` library, the script downloads the dataset ZIP file from the specified URL. The `get` method of `requests` fetches the ZIP file, and the content is stored in `zip_content`.

3. **Extract the ZIP File**: The `zipfile.ZipFile` function reads the downloaded ZIP file directly from memory using `io.BytesIO(zip_content)`. This method is memory efficient as it avoids saving the ZIP file to disk before extraction. The `extractall` method then extracts the contents of the ZIP file to the specified directory (`/content/maestro-v2.0.0`). This directory should be adjusted based on the desired extraction location in your environment.

By automating the download and extraction process, this script streamlines the setup phase for projects utilizing the Maestro dataset, allowing researchers and developers to focus on their analytical and generative tasks without manual data management overhead.


In [ ]:
import requests
import zipfile
import io

# URL of the Maestro dataset
url = 'https://storage.googleapis.com/magentadata/datasets/maestro/v2.0.0/maestro-v2.0.0-midi.zip'

# Download the ZIP file
print("Downloading Maestro dataset...")
response = requests.get(url)
zip_content = response.content

# Extract the ZIP file
print("Extracting Maestro dataset...")
zip_file = zipfile.ZipFile(io.BytesIO(zip_content))
zip_file.extractall("/content/maestro-v2.0.0")  # Adjust this path as necessary


This script performs the extraction of melody and chord sequences from MIDI files in the Maestro dataset and prepares the data for machine learning model training:

1. **MIDI File Processing**: The `process_midi` function reads a MIDI file, extracts the notes, and sorts them by start time and pitch. It then separates the highest note at each time step as the melody and the remaining notes as chords.

2. **Dataset Iteration**: The script iterates through the MIDI files in the Maestro dataset directory using `os.walk`. For each file, it calls `process_midi` to extract melody and chord notes, appending them to `all_melodies` and `all_chords` lists, respectively.

3. **Early Inspection**: After processing the first directory (or a single file for quick inspection), the script prints the collected melody and chord data to provide an insight into the extracted information.

4. **Data Preparation**: Melody sequences are normalized and both melody and chord sequences are transformed into a consistent format suitable for training. The `create_sequences` function generates subsequences of a specified length from the melody data, and `pad_sequences` from Keras standardizes the length of chord sequences.

5. **Zero Padding Analysis**: The script calculates and prints the percentage of zero values (silent timesteps) in the chord sequences at each timestep, giving insight into the sparsity of the data, which can be crucial for model tuning and training efficiency.

By executing these steps, the script not only processes MIDI files for melody and chord extraction but also prepares the data for machine learning, ensuring it is in a suitable format for sequence prediction tasks common in music generation models.


In [ ]:
import os
import pretty_midi
from tqdm import tqdm
from keras.preprocessing.sequence import pad_sequences

def process_midi(file_path):
    """Extracts melodies and chords from a MIDI file.

    Args:
        file_path: The path to the MIDI file.

    Returns:
        A tuple: (melody_notes, chord_notes)
        * melody_notes: A list of extracted melody notes (pitch, start, end)
        * chord_notes: A list of extracted chords (lists of pitches)
    """

    pm = pretty_midi.PrettyMIDI(file_path)

    melody_notes = []
    chord_notes = []

    for instrument in pm.instruments:
        notes = instrument.notes
        notes.sort(key=lambda note: (note.start, -note.pitch))  # Sort by start time, then highest pitch

        for note in notes:
            is_melody = note == notes[0]
            if is_melody:
                melody_notes.append((note.pitch, note.start, note.end))
            else:
                chord_notes.append(note.pitch)

    return melody_notes, chord_notes

# Assuming the Maestro dataset is extracted at this location
dataset_path = '/content/maestro-v2.0.0/maestro-v2.0.0'

all_melodies = []
all_chords = []

# Iterate through the Maestro dataset
for root, dirs, files in os.walk(dataset_path):
    for file_name in tqdm(files, desc="Processing MIDI files"):
        if file_name.endswith(('.midi', '.mid')):
            full_path = os.path.join(root, file_name)

            try:
                melody_notes, chord_notes = process_midi(full_path)
                all_melodies.extend(melody_notes)
                all_chords.extend(chord_notes)

            except Exception as e:
                print(f"Error processing {full_path}: {e}")

            print("Data after processing the first directory:")
            print("Melodies:", all_melodies)  # Inspect the lengths of melody sequences
            print("Chords:", all_chords)
            break

# After the loop completes, you'll have your all_melodies and all_chords lists populated!
# ... (Maestro processing loop) ...

# Data Preparation
melody_sequences = create_sequences(normalize_pitches(all_melodies), seq_length=32)  # Adjust seq_length
chord_sequences = pad_sequences(chord_sequences, maxlen=32, padding='post')

zero_counts = np.zeros((chord_sequences.shape[1],))
for sequence in chord_sequences:
    zero_counts += np.sum(np.all(sequence == 0, axis=1))

# Calculate percentages
zero_percentages = (zero_counts / len(chord_sequences)) * 100

print("Percentage of zeros per timestep:")
for i, p in enumerate(zero_percentages):
    print(f"Timestep {i+1}: {p:.2f}%")

In [ ]:
print(f"Extracted Melody from {file_name}: {melody_notes}")
print(f"Extracted Chords from {file_name}: {chord_notes}")


The code snippet is focused on finalizing the data preparation phase for a machine learning model in music generation. Here’s what each part accomplishes:

1. **Sequence Generation**:
   - `create_sequences` function is called for both `all_melodies` and `all_chords`, with a specified `seq_length` of 32. This function generates sequences from the list of notes, where each sequence has a fixed length, ensuring a consistent input size for the model.
   - `normalize_pitches` is applied to the melodies and chords to scale the pitch values between 0 and 1, enhancing the model’s training process by providing standardized input.

2. **One-hot Encoding**:
   - `to_one_hot` function is defined to convert the numerical chord data into one-hot encoded vectors, which are used as labels for the model training. One-hot encoding is crucial for categorical output like chords, as it allows the model to treat each chord type as a distinct category without any implied ordinal relationship.
   - Inside the `to_one_hot` function, a zero matrix is created with dimensions corresponding to the sequence length and the number of chord classes. It then marks the appropriate positions in the matrix with 1’s based on the chord indices in each sequence, resulting in a one-hot encoded representation of the chord sequences.

3. **Vectorization**:
   - The one-hot encoding process transforms the list of chord indices into a 3D NumPy array, where each row corresponds to a timestep in the sequence, and each column represents a chord class. This format is compatible with the input requirements of many neural network architectures, especially those dealing with multi-class classification tasks.

This preparation ensures that the data is in the correct format for training a neural network, where the model is expected to predict a chord (as a one-hot encoded vector) for each sequence of melody notes.


In [ ]:


# Data Preparation
melody_sequences = create_sequences(normalize_pitches(all_melodies), seq_length=32)  # Adjust seq_length
chord_sequences = create_sequences(normalize_pitches(all_chords), seq_length=32)

import numpy as np

#def to_one_hot(chord_indices, num_classes):
def to_one_hot(chord_sequences, num_classes):
    """Converts sequences of chord indices into one-hot encoded vectors."""
    result = []
    for sequence in chord_sequences:
        one_hot_vectors = np.zeros((len(sequence), num_classes))
        one_hot_vectors[np.arange(len(sequence)), sequence] = 1
        result.append(one_hot_vectors)
    return np.array(result)



The code snippet is designed to set up a neural network model in TensorFlow for predicting chords in music generation. Here’s how it operates:

1. **Unique Chords Calculation**:
   - The `flattened_chords` list is created by flattening `chord_sequences`, which are likely sequences of chords used in the training dataset. This step is crucial for understanding the diversity of the chord vocabulary in the data.
   - `np.unique` function is applied to find all unique chords in the dataset, and `num_unique_chords` is calculated to determine the number of unique chord categories. This information is critical for the model’s output layer to classify into the correct number of chord categories.

2. **Model Definition**:
   - A `Sequential` model is defined using TensorFlow’s Keras API, indicating that the layers in the model will be arranged in sequence.
   - While the code for adding LSTM layers is not shown (`# ... (your LSTM layers)`), it implies that one or more LSTM layers are expected to precede the final dense layer. LSTM layers are instrumental in capturing the temporal dependencies in the sequence of melody notes.
   - The final layer of the model is a `Dense` layer with `num_unique_chords` units, which corresponds to the total number of unique chords found in the dataset. The `softmax` activation function suggests that the model’s output can be interpreted as a probability distribution over all possible chord classes, making it suitable for classification tasks.

3. **Compilation and Training** (implied but not shown):
   - After defining the model architecture, it would typically be compiled with an optimizer and loss function suitable for classification (e.g., `categorical_crossentropy` for multi-class classification), and then trained on preprocessed and sequence-transformed melody and chord data.

This setup ensures that the neural network model is appropriately configured to learn from the sequences of melody data to predict corresponding chords, with an output layer fine-tuned to the specific number of chord variations present in the training dataset.


In [ ]:
import tensorflow as tf
import numpy as np

# ... (rest of your code)

# Calculate the number of unique chords
flattened_chords = [chord for chord_sequence in chord_sequences for chord in chord_sequence]
unique_chords = np.unique(flattened_chords)
num_unique_chords = len(unique_chords)

# Define the model
model = tf.keras.Sequential([
    # ... (your LSTM layers)

    tf.keras.layers.Dense(num_unique_chords, activation='softmax')
])

# ... (rest of your code)


In [ ]:
import tensorflow as tf
import numpy as np

# ... (Your code for calculating num_unique_chords)

print(chord_sequences.shape) # Print the shape here
# One-hot encode chord indices
chord_sequences = to_one_hot(chord_sequences, num_unique_chords)

# Reshape to add features dimension
chord_sequences = chord_sequences.reshape(-1, 32, num_unique_chords)
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(128, return_sequences=True, input_shape=(32, num_unique_chords)),
    tf.keras.layers.LSTM(64),

    # Output layer - Adjust based on chord representation
    tf.keras.layers.Dense(num_unique_chords, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam')

model.summary()

This section of the notebook demonstrates preparing the chord data for a neural network model and defining the model architecture for chord prediction in music generation. The process unfolds as follows:

1. **Inspecting Chord Sequences Shape**:
   - `print(chord_sequences.shape)` is used to display the shape of `chord_sequences` before transformation. This is helpful for understanding the data's structure and ensuring that the one-hot encoding process will be applied correctly.

2. **One-hot Encoding of Chord Sequences**:
   - The `to_one_hot` function is applied to `chord_sequences` with `num_unique_chords` as the total number of chord categories. This converts the chord indices into one-hot encoded vectors, which are suitable for categorical classification tasks in machine learning.

3. **Reshaping Data**:
   - The one-hot encoded `chord_sequences` are reshaped to fit the input requirements of the LSTM layers in the neural network. The reshape operation ensures that each sequence has a specified length (`32` in this case) and a feature dimension equal to `num_unique_chords`, which represents the one-hot encoded vectors.

4. **Neural Network Model Construction**:
   - A `Sequential` model is defined using TensorFlow's Keras API, indicating a linear stack of layers.
   - Two LSTM layers are specified:
        - The first LSTM layer has `128` units and `return_sequences=True`, which ensures that the output is a sequence, feeding into the next LSTM layer.
        - The second LSTM layer has `64` units and compacts the sequential information into a single vector.
   - The output layer is a `Dense` layer with a size equal to `num_unique_chords` and uses a `softmax` activation function. This setup is ideal for classification tasks where the output represents a probability distribution over the chord categories.

5. **Model Compilation**:
   - The model is compiled with `categorical_crossentropy` as the loss function, suitable for multi-class classification tasks, and `adam` as the optimizer, known for its efficiency in handling sparse gradients and adaptive learning rates.

6. **Model Summary**:
   - `model.summary()` provides a textual representation of the neural network architecture, including the configuration of each layer and the total number of parameters that will be trained.

In this code block, the preparation of chord data and the definition of the neural network are crucial steps toward training a model capable of predicting chord sequences that harmonize with given melodies.


In [ ]:
import tensorflow as tf
import numpy as np

# ... (Your code for loading data, creating 'chord_sequences', and calculating 'num_unique_chords')

# ... (Your model definition code - which you already have)

# Prepare Targets (Assuming predicting the next chord)
def prepare_targets(chord_sequences):
    targets = []
    for seq in chord_sequences:
        targets.append(seq[1])  # Take the second chord of each sequence
    return np.array(targets)



targets = prepare_targets(chord_sequences)

print("Shape of chord_sequences:", chord_sequences.shape)
print("Shape of targets:", targets.shape)

print("First 10 sequences:", chord_sequences[:10])
print("First 10 targets:", targets[:10])

print(chord_sequences.shape)
print(targets.shape)


# Model Training
model.fit(chord_sequences, targets, epochs=10, batch_size=32)


In [ ]:
import matplotlib.pyplot as plt

history = model.history  # Assuming you store training results in a 'history' variable
epochs = range(1, len(history.history['loss']) + 1)
loss_values = history.history['loss']

plt.plot(epochs, loss_values, 'o-')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss over Epochs')
plt.show()


The graph shows a plot of training loss over epochs. It illustrates that the loss decreases significantly as the number of epochs increases, indicating that the model is learning and improving its predictions over time. The plot shows a typical learning curve where the rapid decrease in loss suggests quick learning in the initial phase, which then gradually stabilizes as the model begins to converge to a minimum loss value.

In [ ]:
def extract_melody(midi_data):
    melody_instrument = midi_data.instruments[0]  # Assuming melody on the first instrument
    melody_notes = []

    for note in melody_instrument.notes:
        melody_notes.append(note.pitch)

    return melody_notes

def preprocess_melody(melody_notes):
    window_size = 3  # Adjustable
    chord_sequences = []

    for i in range(len(melody_notes) - window_size + 1):
        window = melody_notes[i:i + window_size]
        possible_chords = identify_chords(window)  # Function to define
        chord_sequences.append(possible_chords)

    return chord_sequences


def build_chord_dictionary():
    chord_dict = {}

    # Major Triads
    for root in range(12):
        chord_dict[(root, root + 4, root + 7)] = f"{chr(root + 65)}"  # Example: C Major

    # Minor Triads
    for root in range(12):
        chord_dict[(root, root + 3, root + 7)] = f"{chr(root + 65)}m"  # Example: Cm

    return chord_dict

chord_dictionary = build_chord_dictionary()
print(chord_dictionary)




 I've created these functions to process MIDI files into a format that can be used for machine learning. The goal is to teach a model how to harmonize melodies with appropriate chords by recognizing patterns in music theory.

In [ ]:


def preprocess_melody(melody_notes):
    window_size = 3
    chord_sequences = []

    for i in range(len(melody_notes) - window_size + 1):
        window = melody_notes[i:i + window_size]
        possible_chords = identify_chords(window)
        encoded_chords = [to_one_hot(chord, num_unique_chords) for chord in possible_chords]
        chord_sequences.append(encoded_chords)

    return chord_sequences

def generate_music(model, seed_sequence, num_steps):
    sequence = seed_sequence.copy()
    current_chord = sequence[-1]

    for _ in range(num_steps):
        melody_window = sequence[-window_size:]  # Extract the melody window
        # Concatenate one-hot encoded melody_window and current_chord
        model_input = np.concatenate(melody_window + [current_chord]).reshape(1, window_size + 1, -1)
        prediction = model.predict(model_input)[0]
        new_chord = sample_chord(prediction)
        sequence = np.append(sequence, new_chord, axis=0)
        current_chord = new_chord

    return sequence




The `predict_chords_for_melody` function is designed to predict chords for a given melody using a trained neural network model. Here's a summary of what the function does:

1. **Input**: The function takes three arguments:
   - `model`: The trained neural network model for chord prediction.
   - `melody_notes`: A list of numerical values representing the melody notes.
   - `window_size`: The size of the melody window used for prediction.

2. **Output**: The function returns a list of predicted chords, where each chord is represented as a one-hot encoded vector.

3. **Iterating over Melody**: The function iterates over the `melody_notes` list, creating overlapping windows of size `window_size`. For each window, it performs the following steps:
   a. Prints the type and content of the window for debugging purposes.
   b. Converts the window of melody notes into a one-hot encoded representation using the `to_one_hot` function and the `num_unique_notes` variable (assumed to be defined elsewhere).
   c. Expands the dimensions of the one-hot encoded window to match the input shape expected by the model.

4. **Prediction**: For each window, the function passes the one-hot encoded melody window to the trained model using `model.predict`. The model then predicts the corresponding chord represented as a one-hot encoded vector.

5. **Appending Predictions**: The predicted chord (one-hot encoded vector) for each window is appended to the `predicted_chords` list.

6. **Return**: After iterating over all windows, the function returns the `predicted_chords` list, which contains the predicted chords (one-hot encoded) for each window of the melody.

In summary, the `predict_chords_for_melody` function takes a trained model, a melody represented as numerical values, and a window size as input. It then iterates over the melody, creating overlapping windows, and uses the trained model to predict the corresponding chords for each window. The predicted chords are represented as one-hot encoded vectors and returned as a list.

In [ ]:
def predict_chords_for_melody(model, melody_notes, window_size):
  """
  Predicts chords for a given melody using a trained model.

  Args:
      model: The trained model for chord prediction.
      melody_notes: A list of numerical values representing the melody notes.
      window_size: The size of the melody window for prediction.

  Returns:
      A list of predicted chords (one-hot encoded) for each window.
  """

  predicted_chords = []
  for i in range(len(melody_notes) - window_size + 1):
    window = melody_notes[i:i + window_size]
    print(type(window), window)  # Debugging line: Check window content
    model_input = np.expand_dims(to_one_hot(window, num_unique_notes), axis=0)  # Assuming one-hot encoded melody
    prediction = model.predict(model_input)[0]
    predicted_chords.append(prediction)

  return predicted_chords

  midi_file_path = '/content/basic_midi_file.midi'  # Adjust if necessary
  melody_notes = extract_melody(midi_file_path)
  num_unique_notes = 12  # Assuming a single octave


The code snippet provided is the culmination of previous steps, where a MIDI file path is specified and used to load MIDI data. The `pretty_midi` library's `PrettyMIDI` class is instantiated with the MIDI file, which then allows for the extraction of melody notes using a custom function `extract_melody`. The melody is then fed into a model that suggests chords to accompany the melody, using the `suggest_chords_for_melody` function.

Here's a detailed explanation of the process:

1. I set the variable `midi_file_path` to the location of a MIDI file that contains the musical piece I'm working with.

2. Using the `pretty_midi` library, I create a `PrettyMIDI` object with this file. This object allows me to interact with the musical data contained within the MIDI file.

3. I call the `extract_melody` function on the `PrettyMIDI` object to isolate the melody from the rest of the musical information. This function is designed to identify and separate the melody based on certain musical properties, such as pitch or the instrument's track.

4. With the melody notes in hand, I use the pre-trained neural network model within the `suggest_chords_for_melody` function. This model has been trained to predict chords that harmonize with given melodies.

5. The `suggest_chords_for_melody` function processes the melody and produces a list of suggested chords that would likely harmonize with each segment of the melody.

6. Finally, I print out the list of suggested chords. This output serves as a guide for creating a harmonized version of the original melody and can be used by musicians for arrangement purposes or further music production work.

In [ ]:
midi_file_path = '/content/basic_midi_file.midi'
midi_data = pretty_midi.PrettyMIDI(midi_file_path)
melody_notes = extract_melody(midi_data)
suggested_chords = suggest_chords_for_melody(model, melody_notes)
print("Suggested Chords:", suggested_chords)




In conclusion, despite the unexpected setback of not being able to generate chords due to some elusive errors, this assignment was a meaningful journey through the intricacies of music generation with AI. Notably, significant accomplishments were made:

1. Successfully installed and utilized key libraries such as `pretty_midi`, setting the foundation for MIDI data manipulation.
2. Gained hands-on experience in processing MIDI files, extracting valuable musical information.
3. Developed a deeper understanding of neural network architectures by constructing a model aimed at music generation.
4. Applied concepts of data preprocessing, normalization, and preparation for model compatibility.
5. Strengthened problem-solving skills, particularly in debugging and iterative code improvement.

While the final goal of chord generation remains unmet, the skills and knowledge acquired lay the groundwork for continued exploration and development in this area. Moving forward, I will persist in debugging the current issues, with a firm belief that a solution is within reach. This perseverance, coupled with the progress made thus far, assures me that success is not a matter of 'if' but 'when'.